# Importing Libraries

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

#nltk.download('all')

# Reading Docs

In [9]:
path = "corpus/"

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        files.append(os.path.join(r, file))

# Removing Headers and HTML Tags, Tokenization, Lower Casing

In [10]:
docs_words = {}
i = 0 #for docs_words

for f in files:
    doc = open(f, errors='ignore')
    html = doc.read()
    
    index = html.find("<html")
    if index == -1 :
        html.find("<Html")
    elif index == -1 : 
        html.find("<HTML")
    html = html[index:]

    soup = BeautifulSoup(html)

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    #print("Doc : " + text + "\n\n")
    tknzr = TweetTokenizer()
    tokens = tknzr.tokenize(text)
    words_pos = {}
    j = 0; #for words_pos
    for t in tokens:
        t = t.lower() #lowering Case
        t = t.replace("'", "")
        if t not in words_pos.keys():
            words_pos[t] = []
        if t in words_pos.keys():
            words_pos[t].append(j)
        j = j + 1
    
    docs_words[i] = words_pos
    i = i + 1

#print(docs_words) 

# Removing stop words

In [11]:
f = open("stoplist.txt", "r") 
stoplist = f.read().splitlines() #Stoplist words

for i in range(len(docs_words)):
    for key in list(docs_words[i]):
        if key in stoplist or len(key) == 1:
            docs_words[i].pop(key,None)
#print(docs_words)            

# Shaping up terms and docs

In [12]:
words_docs = {}

for i in range(len(docs_words)):
    for key in docs_words[i].keys():
        docs_pos = {}
        docs_pos[i] = docs_words[i].get(key)
        if key not in words_docs.keys():
            words_docs[key] = docs_pos
        elif key in words_docs.keys():
             words_docs[key][i] = docs_pos[i]

#print(words_docs)        

# Stemming

In [13]:
ps = PorterStemmer()

for key in list(words_docs):
    stem = ps.stem(key)
    if stem != key:
        if stem not in words_docs.keys():
            words_docs[stem] = words_docs.pop(key)
        elif stem in words_docs.keys():
            dictn = words_docs.pop(key)
            for doc in dictn.keys():
                if doc not in words_docs[stem].keys():
                    words_docs[stem][doc] = dictn[doc]
                elif doc in words_docs[stem].keys():
                    words_docs[stem][doc] = words_docs[stem][doc] + dictn[doc]

#print(docs_words)

# Writing docids.txt 

In [14]:
f_docs = open("docids.txt", 'w')
i = 0
for f in files:
    doc_name = os.path.basename(f)
    f_docs.write(str(i) + "\t" + doc_name + "\n")
    i = i + 1

# Writing termids.txt

In [15]:
f_terms = open("termids.txt", 'w', errors="ignore")
id = 0

for key in words_docs.keys():
    f_terms.write(str(id) + "\t" + key + "\n")
    id = id + 1

In [17]:
print(words_docs["appl"])

{25: [4187], 26: [4187], 102: [722], 171: [4081], 234: [1325, 196, 476, 508, 2533], 316: [114, 5366], 322: [7212], 366: [109], 425: [2619], 444: [223], 511: [5003], 529: [612], 547: [2453], 554: [91], 571: [2558], 579: [929], 580: [929], 582: [929], 583: [929], 586: [929], 587: [929], 588: [929], 591: [929], 592: [929], 595: [108, 2037], 612: [929], 614: [929], 616: [929], 678: [15027, 15071], 699: [65], 782: [784], 846: [111, 4521], 1032: [3237], 1046: [3620], 1050: [1068, 702, 1041], 1059: [2294, 2537, 2206], 1060: [76, 273, 503, 689, 1038, 1099, 1391, 685], 1072: [933, 1474], 1288: [51, 188, 204], 1318: [76, 273, 503, 1046, 1106, 1398], 1384: [2630], 1386: [112348], 1391: [3141], 1400: [3282], 1404: [5045, 5187, 5295], 1405: [5545, 5687, 5795], 1409: [3454], 1414: [2735], 1415: [3276], 1417: [2939], 1637: [3835, 3320, 3826], 1653: [107, 4908], 1656: [80, 775, 881, 902, 1120, 1557], 1657: [80, 775, 881, 902, 1120, 1557], 1661: [32], 1700: [1720], 1709: [504, 423], 1794: [98, 226, 417